In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data = pd.read_csv('50_Startups.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [15]:
data.State.unique()

array(['New York', 'California', 'Florida'], dtype=object)

In [4]:
data.describe()

R&D Spend  Administration  Marketing Spend         Profit
count      50.000000       50.000000        50.000000      50.000000
mean    73721.615600   121344.639600    211025.097800  112012.639200
std     45902.256482    28017.802755    122290.310726   40306.180338
min         0.000000    51283.140000         0.000000   14681.400000
25%     39936.370000   103730.875000    129300.132500   90138.902500
50%     73051.080000   122699.795000    212716.240000  107978.190000
75%    101602.800000   144842.180000    299469.085000  139765.977500
max    165349.200000   182645.560000    471784.100000  192261.830000

In [5]:
#Features and label
features = data.iloc[:,:-1].values
label = data.iloc[:,[-1]].values

In [8]:
#Deal with categorical data
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
stateOHE = ohe.fit_transform(features[:,3].reshape(-1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
finalFeatures = np.concatenate((stateOHE,features[:,[0,1,2]]), axis=1 )
finalFeatures

array([[0.0, 0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [0.0, 1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [1.0, 0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [1.0, 0.0, 0.0, 123334.88, 108679.17, 304981.62],
       [0.0, 1.0, 0.0, 101913.08, 110594.11, 229160.95],
       [1.0, 0.0, 0.0, 100671.96, 91790.61, 249744.55],
       [0.0, 1.0, 0.0, 93863.75, 127320.38, 249839.44],
       [1.0, 0.0, 0.0, 91992.39, 135495.07, 252664.93],
       [0.0, 1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [1.0, 0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [0.0, 0.0, 1.0, 94657.16, 145077.58

In [11]:
#Train test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(finalFeatures, label, test_size=0.2, random_state=10)

In [12]:
#Model

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [14]:
#Quaality
# SL = 0.05
# CL = 0.95

print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.9385918220043519
0.9901105113397705


In [19]:
state="California"
if state in ohe.categories_[0]:
  print("Pass")
else:
  print("Doesnt exists")

Pass


In [20]:
ohe.transform(np.array([["California"]]))

array([[1., 0., 0.]])

In [21]:
ohe.inverse_transform(np.array([[1., 0., 0.]]))

array([['California']], dtype=object)

In [23]:
#Deployment App Test

rdSpend = float(input("Enter RD Spend: "))
admSpend = float(input("Enter Admin Spend: "))
markSpend = float(input("Enter Marketing Spend: "))
state = input("Enter State")


if state in ohe.categories_[0]:
  dummyVar = ohe.transform(np.array([[state]]))
  finalFeature = np.concatenate((dummyVar,np.array([[rdSpend,admSpend,markSpend]])), axis= 1)
  profit = model.predict(finalFeature)
  print("Predicted profit is $ {}".format(profit))
else:
  print("Model cant predict for {} state".format(state))


Enter RD Spend: 453345
Enter Admin Spend: 897453
Enter Marketing Spend: 2332455
Enter StateCalifornia
Predicted profit is $ [[452506.15043366]]


In [24]:
#Final Deploy

pickle.dump(model,open("ProfitPredictor.mdl","wb"))
pickle.dump(ohe,open("StateEncoderDecoder.obj","wb"))

In [ ]:
#